In [1]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("mini-contest/dataset-agent.csv")

# Display the first few rows of the DataFrame
print(df.head(1))

   id                                          project_a  \
0   2  https://github.com/prettier-solidity/prettier-...   

                                    project_b  weight_a  weight_b  \
0  https://github.com/nomicfoundation/hardhat  0.101669  0.898331   

   agent_weight_a  agent_weight_b  
0            0.85            0.15  


In [3]:
from deepfunding import run_comparison
from tools.data_collection import fetch_repo_metrics

# Initialize agent weight columns if they don't exist
if 'agent_weight_a' not in df.columns:
    df['agent_weight_a'] = None
if 'agent_weight_b' not in df.columns:
    df['agent_weight_b'] = None

for i in range(len(df)):
    if pd.notnull(df.at[i, 'agent_weight_a']) and pd.notnull(df.at[i, 'agent_weight_b']):
        continue

    repo_a_url = df.iloc[i]["project_a"]
    repo_b_url = df.iloc[i]["project_b"]
    
    repo_a = { "url": repo_a_url, **fetch_repo_metrics(repo_a_url) }
    repo_b = { "url": repo_b_url, **fetch_repo_metrics(repo_b_url) }

    agent_result = run_comparison(repo_a, repo_b)
    
    df.at[i, 'agent_weight_a'] = agent_result['weights'][repo_a_url]
    df.at[i, 'agent_weight_b'] = agent_result['weights'][repo_b_url]
    
    print(f"{repo_a_url} VS {repo_b_url}")
    print(f"Agent: {df.iloc[i]['agent_weight_a']} {df.iloc[i]['agent_weight_b']}")
    print(f"Human: {df.iloc[i]['weight_a']} {df.iloc[i]['weight_b']}")

Skipping row 0:
id                                                                2
project_a         https://github.com/prettier-solidity/prettier-...
project_b                https://github.com/nomicfoundation/hardhat
weight_a                                                   0.101669
weight_b                                                   0.898331
agent_weight_a                                                 0.85
agent_weight_b                                                 0.15
Name: 0, dtype: object
Skipping row 1:
id                                                                3
project_a         https://github.com/prettier-solidity/prettier-...
project_b                         https://github.com/consensys/teku
weight_a                                                   0.669446
weight_b                                                   0.330554
agent_weight_a                                                 0.35
agent_weight_b                                               

KeyboardInterrupt: 

In [4]:
# Save the updated DataFrame back to CSV
print(df.head(5))
df.to_csv("mini-contest/dataset-agent.csv", index=False)


   id                                          project_a  \
0   2  https://github.com/prettier-solidity/prettier-...   
1   3  https://github.com/prettier-solidity/prettier-...   
2   4  https://github.com/prettier-solidity/prettier-...   
3   5  https://github.com/prettier-solidity/prettier-...   
4   6  https://github.com/prettier-solidity/prettier-...   

                                    project_b  weight_a  weight_b  \
0  https://github.com/nomicfoundation/hardhat  0.101669  0.898331   
1           https://github.com/consensys/teku  0.669446  0.330554   
2        https://github.com/ethereum/solidity  0.449022  0.550978   
3   https://github.com/ethereum/remix-project  0.498396  0.501604   
4     https://github.com/ethereum/go-ethereum  0.272503  0.727497   

   agent_weight_a  agent_weight_b  
0            0.85            0.15  
1            0.35            0.65  
2            0.15            0.85  
3            0.20            0.80  
4            0.15            0.85  
